# NOTE:  THIS NOTEBOOK WILL TAKE ABOUT 30 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.

# Fine-Tuning a BERT Model and Create a Text Classifier

In the previous section, we've already performed the Feature Engineering to create BERT embeddings from the `reviews_body` text using the pre-trained BERT model, and split the dataset into train, validation and test files. To optimize for Tensorflow training, we saved the files in TFRecord format. 

Now, let’s fine-tune the BERT model to our Customer Reviews Dataset and add a new classification layer to predict the `star_rating` for a given `review_body`.


As mentioned earlier, BERT’s attention mechanism is called a Transformer. This is, not coincidentally, the name of the popular BERT Python library, “Transformers,” maintained by a company called HuggingFace. We will use a variant of BERT called [DistilBert](https://arxiv.org/pdf/1910.01108.pdf) which requires less memory and compute, but maintains very good accuracy on our dataset.

In [1]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

# _PRE-REQUISITE: You need to have succesfully run the notebooks in the `PREPARE` section before you continue with this notebook._

In [2]:
%store -r processed_train_data_s3_uri

In [3]:
print(processed_train_data_s3_uri)

s3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/output/bert-train


In [4]:
%store -r processed_validation_data_s3_uri

In [5]:
print(processed_validation_data_s3_uri)

s3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/output/bert-validation


In [6]:
%store -r processed_test_data_s3_uri

In [7]:
print(processed_test_data_s3_uri)

s3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/output/bert-test


In [8]:
%store -r max_seq_length

In [9]:
print(max_seq_length)

64


In [10]:
%store -r experiment_name

In [11]:
print(experiment_name)

Amazon-Customer-Reviews-BERT-Experiment-1618247311


In [12]:
%store -r trial_name

In [13]:
print(trial_name)

trial-1618247314


# Specify the Dataset in S3
We are using the train, validation, and test splits created in the previous section.

In [14]:
print(processed_train_data_s3_uri)

!aws s3 ls $processed_train_data_s3_uri/

s3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/output/bert-train
2021-04-12 17:19:49   10478979 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2021-04-12 17:19:49    2313103 part-algo-1-amazon_reviews_us_Gift_Card_v1_00.tfrecord
2021-04-12 17:20:24   11706723 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [15]:
print(processed_validation_data_s3_uri)

!aws s3 ls $processed_validation_data_s3_uri/

s3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/output/bert-validation
2021-04-12 17:19:50     582980 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2021-04-12 17:19:50     128566 part-algo-1-amazon_reviews_us_Gift_Card_v1_00.tfrecord
2021-04-12 17:20:25     651334 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [16]:
print(processed_test_data_s3_uri)

!aws s3 ls $processed_test_data_s3_uri/

s3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/output/bert-test
2021-04-12 17:19:50     581539 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2021-04-12 17:19:50     128821 part-algo-1-amazon_reviews_us_Gift_Card_v1_00.tfrecord
2021-04-12 17:20:25     651069 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


# Specify S3 `Distribution Strategy`

In [17]:
from sagemaker.inputs import TrainingInput

s3_input_train_data = TrainingInput(s3_data=processed_train_data_s3_uri, distribution="ShardedByS3Key")
s3_input_validation_data = TrainingInput(s3_data=processed_validation_data_s3_uri, distribution="ShardedByS3Key")
s3_input_test_data = TrainingInput(s3_data=processed_test_data_s3_uri, distribution="ShardedByS3Key")

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/output/bert-train', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/output/bert-validation', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/output/bert-test', 'S3DataDistributionType': 'ShardedByS3Key'}}}


# Setup Hyper-Parameters for Classification Layer

In [18]:
print(max_seq_length)

64


In [19]:
epochs = 3
learning_rate = 0.00001
epsilon = 0.00000001
train_batch_size = 128
validation_batch_size = 128
test_batch_size = 128
train_steps_per_epoch = 100
validation_steps = 100
test_steps = 100
train_instance_count = 1
train_instance_type = "ml.c5.9xlarge"
train_volume_size = 1024
use_xla = True
use_amp = True
freeze_bert_layer = False
enable_sagemaker_debugger = True
enable_checkpointing = False
enable_tensorboard = True
input_mode = "File"
run_validation = True
run_test = True
run_sample_predictions = True

# Setup Metrics To Track Model Performance

These sample log lines...
```
45/50 [=====>..] - ETA: 3s - loss: 0.425 - accuracy: 0.881
50/50 [=======>] - ETA: 0s - val_loss: 0.407 - val_accuracy: 0.885
```
...will produce the following 4 metrics in CloudWatch:

`loss` = 0.425

`accuracy` = 0.881

`val_loss` = 0.407

`val_accuracy` = 0.885

In [20]:
metrics_definitions = [
    {"Name": "train:loss", "Regex": "loss: ([0-9\\.]+)"},
    {"Name": "train:accuracy", "Regex": "accuracy: ([0-9\\.]+)"},
    {"Name": "validation:loss", "Regex": "val_loss: ([0-9\\.]+)"},
    {"Name": "validation:accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
]

# Setup SageMaker Debugger
Define Debugger Rules as deccribed here:  https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html

In [21]:
from sagemaker.debugger import Rule
from sagemaker.debugger import rule_configs
from sagemaker.debugger import ProfilerRule
from sagemaker.debugger import CollectionConfig
from sagemaker.debugger import DebuggerHookConfig

actions = rule_configs.ActionList(
    #    rule_configs.StopTraining(),
    #    rule_configs.Email("")
)

rules = [
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),    
#     ProfilerRule.sagemaker(rule_configs.BatchSize()),
#     ProfilerRule.sagemaker(rule_configs.CPUBottleneck()),
#     ProfilerRule.sagemaker(rule_configs.GPUMemoryIncrease()),
#     ProfilerRule.sagemaker(rule_configs.IOBottleneck()),
#     ProfilerRule.sagemaker(rule_configs.LoadBalancing()),
#     ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
#     ProfilerRule.sagemaker(rule_configs.OverallSystemUsage()),
#     ProfilerRule.sagemaker(rule_configs.StepOutlier()),
#     Rule.sagemaker(
#         base_config=rule_configs.loss_not_decreasing(),
#         rule_parameters={
#             "collection_names": "losses,metrics",
#             "use_losses_collection": "true",
#             "num_steps": "10",
#             "diff_percent": "50",
#         },
#         collections_to_save=[
#             CollectionConfig(
#                 name="losses",
#                 parameters={
#                     "save_interval": "10",
#                 },
#             ),
#             CollectionConfig(
#                 name="metrics",
#                 parameters={
#                     "save_interval": "10",
#                 },
#             ),
#         ],
#         actions=actions,
#     ),
#     Rule.sagemaker(
#         base_config=rule_configs.overtraining(),
#         rule_parameters={
#             "collection_names": "losses,metrics",
#             "patience_train": "10",
#             "patience_validation": "10",
#             "delta": "0.5",
#         },
#         collections_to_save=[
#             CollectionConfig(
#                 name="losses",
#                 parameters={
#                     "save_interval": "10",
#                 },
#             ),
#             CollectionConfig(
#                 name="metrics",
#                 parameters={
#                     "save_interval": "10",
#                 },
#             ),
#         ],
#         actions=actions,
#     )    
]

hook_config = DebuggerHookConfig(
    hook_parameters={
        "save_interval": "10",  # number of steps
        "export_tensorboard": "true",
        "tensorboard_dir": "hook_tensorboard/",
    }
)

## Specify a Debugger profiler configuration

The following configuration will capture system metrics at 500 milliseconds. The system metrics include utilization per CPU, GPU, memory utilization per CPU, GPU as well I/O and network.

Debugger will capture detailed profiling information from step 5 to step 15. This information includes Horovod metrics, dataloading, preprocessing, operators running on CPU and GPU.

In [22]:
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(local_path="/opt/ml/output/profiler/", start_step=5, num_steps=10),
)

# Specify Checkpoint S3 Location
This is used for Spot Instances Training.  If nodes are replaced, the new node will start training from the latest checkpoint.

In [23]:
import uuid

checkpoint_s3_prefix = "checkpoints/{}".format(str(uuid.uuid4()))
checkpoint_s3_uri = "s3://{}/{}/".format(bucket, checkpoint_s3_prefix)

print(checkpoint_s3_uri)

s3://sagemaker-us-east-1-117859797117/checkpoints/fab42251-467c-46a9-bfc6-8fa60a8276b1/


# Setup Our BERT + TensorFlow Script to Run on SageMaker
Prepare our TensorFlow model to run on the managed SageMaker service

In [24]:
!pygmentize src/tf_bert_reviews.py

import time
import random
import pandas as pd
from glob import glob
import pprint
import argparse
import json
import subprocess
import sys
import os
import csv

# subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0'])
import tensorflow as tf
import pandas as pd
import numpy as np

subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers==3.5.1"])
# subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker-tensorflow==2.1.0.1.0.0'])
# subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'smdebug==0.9.3'])
subprocess.check_call([sys.executable, "-m", "pip", "install", "scikit-learn==0.23.1"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "matplotlib==3.2.1"])

from transformers import DistilBertTokenizer
from transformers import DistilBertConfig
from transformers import TFDistilBertForSequenceClassification

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models impor

In [25]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point="tf_bert_reviews.py",
    source_dir="src",
    role=role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    volume_size=train_volume_size,
    checkpoint_s3_uri=checkpoint_s3_uri,
    py_version="py37",
    framework_version="2.3.1",
    hyperparameters={
        "epochs": epochs,
        "learning_rate": learning_rate,
        "epsilon": epsilon,
        "train_batch_size": train_batch_size,
        "validation_batch_size": validation_batch_size,
        "test_batch_size": test_batch_size,
        "train_steps_per_epoch": train_steps_per_epoch,
        "validation_steps": validation_steps,
        "test_steps": test_steps,
        "use_xla": use_xla,
        "use_amp": use_amp,
        "max_seq_length": max_seq_length,
        "freeze_bert_layer": freeze_bert_layer,
        "enable_sagemaker_debugger": enable_sagemaker_debugger,
        "enable_checkpointing": enable_checkpointing,
        "enable_tensorboard": enable_tensorboard,
        "run_validation": run_validation,
        "run_test": run_test,
        "run_sample_predictions": run_sample_predictions,
    },
    input_mode=input_mode,
    metric_definitions=metrics_definitions,
    rules=rules,
    debugger_hook_config=hook_config,
    profiler_config=profiler_config,
)

# Create the `Experiment Config`

In [26]:
experiment_config = {"ExperimentName": experiment_name, "TrialName": trial_name, "TrialComponentDisplayName": "train"}

# Train the Model on SageMaker

In [27]:
estimator.fit(
    inputs={"train": s3_input_train_data, "validation": s3_input_validation_data, "test": s3_input_test_data},
    experiment_config=experiment_config,
    wait=False,
)

In [28]:
training_job_name = estimator.latest_training_job.name
print("Training Job Name:  {}".format(training_job_name))

Training Job Name:  tensorflow-training-2021-04-12-21-49-31-083


In [29]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(
            region, training_job_name
        )
    )
)

In [30]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, training_job_name
        )
    )
)

In [31]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(
            bucket, training_job_name, region
        )
    )
)

In [32]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Checkpoint Data</a> After The Training Job Has Completed</b>'.format(
            bucket, checkpoint_s3_prefix, region
        )
    )
)

In [33]:
%%time

estimator.latest_training_job.wait(logs=False)


2021-04-12 21:49:31 Starting - Starting the training job
2021-04-12 21:49:34 Starting - Launching requested ML instances..................
2021-04-12 21:51:09 Starting - Preparing the instances for training..........
2021-04-12 21:52:07 Downloading - Downloading input data.
2021-04-12 21:52:16 Training - Downloading the training image...
2021-04-12 21:52:38 Training - Training image download completed. Training in progress..........................................................................................................................................................................................................................................................................................................................................................................................................................................
2021-04-12 22:28:18 Uploading - Uploading generated training model.....................
2021-04-12 22:30:07 Failed - Training job failed


UnexpectedStatusException: Error for Training job tensorflow-training-2021-04-12-21-49-31-083: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/local/bin/python3.7 tf_bert_reviews.py --enable_checkpointing False --enable_sagemaker_debugger True --enable_tensorboard True --epochs 3 --epsilon 1e-08 --freeze_bert_layer False --learning_rate 1e-05 --max_seq_length 64 --model_dir s3://sagemaker-us-east-1-117859797117/tensorflow-training-2021-04-12-21-49-31-083/model --run_sample_predictions True --run_test True --run_validation True --test_batch_size 128 --test_steps 100 --train_batch_size 128 --train_steps_per_epoch 100 --use_amp True --use_xla True --validation_batch_size 128 --validation_steps 100"
2021-04-12 21:52:42.075286: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-04-12 21:52:42.102211: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
WARNING:tensorf

# Wait Until the ^^ Training Job ^^ Completes Above!

In [34]:
%store training_job_name

Stored 'training_job_name' (str)


In [35]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

download: s3://sagemaker-us-east-1-117859797117/tensorflow-training-2021-04-12-21-49-31-083/output/model.tar.gz to ./model.tar.gz


In [36]:
!mkdir -p ./model/
!tar -xvzf ./model.tar.gz -C ./model/

transformers/
transformers/fine-tuned/
transformers/fine-tuned/tf_model.h5
transformers/fine-tuned/config.json
tensorboard/
tensorboard/validation/
tensorboard/validation/events.out.tfevents.1618265901.ip-10-0-151-4.ec2.internal.89.29704.v2
tensorboard/validation/events.out.tfevents.1618264839.ip-10-0-151-4.ec2.internal.89.25864.v2
tensorboard/train/
tensorboard/train/events.out.tfevents.1618265900.ip-10-0-151-4.ec2.internal.89.29684.v2
tensorboard/train/plugins/
tensorboard/train/plugins/profile/
tensorboard/train/plugins/profile/2021_04_12_21_54_26/
tensorboard/train/plugins/profile/2021_04_12_21_54_26/ip-10-0-151-4.ec2.internal.overview_page.pb
tensorboard/train/plugins/profile/2021_04_12_21_54_26/ip-10-0-151-4.ec2.internal.memory_profile.json.gz
tensorboard/train/plugins/profile/2021_04_12_21_54_26/ip-10-0-151-4.ec2.internal.trace.json.gz
tensorboard/train/plugins/profile/2021_04_12_21_54_26/ip-10-0-151-4.ec2.internal.tensorflow_stats.pb
tensorboard/train/plugins/profile/2021_04_12

In [37]:
!saved_model_cli show --all --dir ./model/tensorflow/saved_model/0/

2021-04-12 22:30:46.933404: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-04-12 22:30:46.933451: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 64)
        name: serving_defau

# Analyze Debugger Rules

In [38]:
estimator.latest_training_job.rule_job_summary()

[{'RuleConfigurationName': 'ProfilerReport',
  'RuleEvaluationJobArn': 'arn:aws:sagemaker:us-east-1:117859797117:processing-job/tensorflow-training-2021-0-profilerreport-ff29bba7',
  'RuleEvaluationStatus': 'InProgress',
  'LastModifiedTime': datetime.datetime(2021, 4, 12, 22, 30, 25, 725000, tzinfo=tzlocal())}]

In [39]:
%store

Stored variables and their in-db values:
balance_dataset                                       -> True
balanced_bias_data_jsonlines_s3_uri                   -> 's3://sagemaker-us-east-1-117859797117/bias-detect
balanced_bias_data_s3_uri                             -> 's3://sagemaker-us-east-1-117859797117/bias-detect
bias_data_s3_uri                                      -> 's3://sagemaker-us-east-1-117859797117/bias-detect
experiment_name                                       -> 'Amazon-Customer-Reviews-BERT-Experiment-161824731
feature_group_name                                    -> 'reviews-feature-group-1618247324'
feature_store_offline_prefix                          -> 'reviews-feature-store-1618247324'
ingest_create_athena_db_passed                        -> True
ingest_create_athena_table_parquet_passed             -> True
ingest_create_athena_table_tsv_passed                 -> True
max_seq_length                                        -> 64
model_ab_endpoint_name             

# Show the Experiment Tracking Lineage

In [40]:
from sagemaker.analytics import ExperimentAnalytics

import pandas as pd

pd.set_option("max_colwidth", 500)

experiment_analytics = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=experiment_name,
    metric_names=["validation:accuracy"],
    sort_by="CreationTime",
    sort_order="Descending",
)

experiment_analytics_df = experiment_analytics.dataframe()
experiment_analytics_df

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,enable_checkpointing,enable_sagemaker_debugger,enable_tensorboard,...,code - MediaType,code - Value,raw-input-data - MediaType,raw-input-data - Value,bert-test - MediaType,bert-test - Value,bert-train - MediaType,bert-train - Value,bert-validation - MediaType,bert-validation - Value
0,tensorflow-training-2021-04-12-21-49-31-083-aws-training-job,train,arn:aws:sagemaker:us-east-1:117859797117:training-job/tensorflow-training-2021-04-12-21-49-31-083,763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.3.1-cpu-py37,1.0,ml.c5.9xlarge,1024.0,false,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tensorflow-training-2021-04-12-20-53-49-447-aws-training-job,train,arn:aws:sagemaker:us-east-1:117859797117:training-job/tensorflow-training-2021-04-12-20-53-49-447,763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.3.1-cpu-py37,1.0,ml.c5.9xlarge,1024.0,false,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sagemaker-scikit-learn-2021-04-12-17-09-10-985-aws-processing-job,prepare,arn:aws:sagemaker:us-east-1:117859797117:processing-job/sagemaker-scikit-learn-2021-04-12-17-09-10-985,NaN,2.0,ml.c5.2xlarge,30.0,NaN,NaN,NaN,...,NaN,s3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/input/code/preprocess-scikit-text-to-bert-feature-store.py,NaN,s3://sagemaker-us-east-1-117859797117/amazon-reviews-pds/tsv/,NaN,s3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/output/bert-test,NaN,s3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/output/bert-train,NaN,s3://sagemaker-us-east-1-117859797117/sagemaker-scikit-learn-2021-04-12-17-09-10-985/output/bert-validation


In [41]:
from sagemaker.lineage.visualizer import LineageTableVisualizer

lineage_table_viz = LineageTableVisualizer(sess)
lineage_table_viz_df = lineage_table_viz.show(training_job_name=training_job_name)
lineage_table_viz_df

,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...2021-04-12-17-09-10-985/output/bert-test,Input,DataSet,ContributedTo,artifact
1,s3://...4-12-17-09-10-985/output/bert-validation,Input,DataSet,ContributedTo,artifact
2,s3://...021-04-12-17-09-10-985/output/bert-train,Input,DataSet,ContributedTo,artifact
3,76310...s.com/tensorflow-training:2.3.1-cpu-py37,Input,Image,ContributedTo,artifact
4,s3://...ts/fab42251-467c-46a9-bfc6-8fa60a8276b1/,Output,Checkpoint,Produced,artifact
